In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML
from astroML.datasets import fetch_LINEAR_sample
from astropy.coordinates import SkyCoord
import astropy.units as u

# Basic libraries
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm

# CONFIGURATION
# -------------
sns.set_theme(style='white') # setting the theme for plotting
sys.path.insert(0,'../src/')
np.random.seed(42)

# configuring plotting colors
colors = ['#465BBB', '#A82F43', '#3F8FCE', '#7ABBCE', '#3A3865', '#612A37', '#DC5433', '#F29457']
b1 = '#465BBB'
b2 = '#3F8FCE'
b3 = '#7ABBCE'
b4 = '#3A3865'
black1 = '#22212A'
black2 = '#2D1E21'
or1 = '#A82F43'
or2 = '#612A37'
or3 = '#DC5433'
or4 = '#F29457'
muted_colors = sns.set_palette(sns.color_palette(colors))

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rcParams['text.latex.preamble'] = r'\usepackage{txfonts}'  # Use TX Times font

# Set up your font properties (optional)
font = FontProperties()
font.set_family('serif')
font.set_name('Times')  # Times font
%matplotlib inline 

In [3]:
#| code-fold: false
# Importing custom libraries
# ----------------------------
sys.path.insert(0,'../src/')
from BE_plotting import*
#from RR_lyrae_params import*
from blazhko_analysis import*

In [7]:
#| code-fold: false
# DATA
dataL = fetch_LINEAR_sample(data_home='../inputs') # fetching the data from astroML data library
end = 'ffinal'

# DATA from previous notebook
BE_STARS = pd.read_csv("../outputs/FINAL_BE_CATALOGUE_EXPANDED.csv")
be_cand = pd.read_csv('../outputs/blazhko_candidates_expanded.csv')
lc_analysis = pd.read_csv("../outputs/LC_ANALYSIS_"+end+".csv")
rrlyrae = pd.read_csv('../outputs/Lrrlyr_unprocessed.csv')
periodogr = pickle.load(open("../outputs/periodograms_"+end+".pkl", "rb"))
fits = np.load("../outputs/fits_"+end+".npy", allow_pickle=True)
ztf_data = np.load("../outputs/ztf_data_filter"+end+".npy", allow_pickle=True)

---

# Selection of Blazhko candidates

In [8]:
lc_main = pd.read_csv('../outputs/lc_MASTER.csv')
lc_main.head()

,LINEAR ID,Plinear,Pztf,N_L,N_Z,L_chi2r,Z_chi2r,L_chi2,Z_chi2,Lampl,Zampl,Ampl_diff,BpeakL,BpeakZ,BperiodL,BperiodZ,Periodogram_f,B_score,Blazhko_f
0,29848,0.557020,0.557040,301,43,1.4,3.5,3.0,12.6,0.56,0.93,0.37,1.8328,1.7982,26.6205,333.3333,-,6,1
1,50402,0.643303,0.643294,284,586,0.7,1.1,0.6,1.8,0.48,0.69,0.21,1.6223,1.5918,14.7351,26.8420,-,0,-1
2,62892,0.530776,0.530785,276,771,0.9,3.2,1.1,19.8,0.62,0.64,0.02,1.9519,1.9433,14.7319,16.8634,-,5,1
3,91437,0.674733,0.674737,177,564,1.3,2.0,2.8,5.6,0.87,1.21,0.34,1.5498,1.4849,14.7580,355.8719,-,0,-1
4,95250,0.313870,0.313876,222,916,0.8,1.4,0.8,3.0,0.48,0.46,0.02,3.2565,3.1889,14.1844,342.4658,-,0,-1


In [9]:
blazhko_can = pd.DataFrame(columns=lc_main.columns)
lids = list(lc_main['LINEAR ID'])

for n, i in enumerate(lids):
    if lc_main.loc[n, 'Blazhko_f'] in [0, 1, 2]:  # Check for valid Blazhko_f values
        row = lc_main.iloc[[n]]  # Extract the row as a DataFrame
        blazhko_can = pd.concat([blazhko_can, row], ignore_index=True)

print(blazhko_can.shape)
blazhko_can.head()

(618, 19)


,LINEAR ID,Plinear,Pztf,N_L,N_Z,L_chi2r,Z_chi2r,L_chi2,Z_chi2,Lampl,Zampl,Ampl_diff,BpeakL,BpeakZ,BperiodL,BperiodZ,Periodogram_f,B_score,Blazhko_f
0,29848,0.557020,0.557040,301,43,1.4,3.5,3.0,12.6,0.56,0.93,0.37,1.8328,1.7982,26.6205,333.3333,-,6,1
1,62892,0.530776,0.530785,276,771,0.9,3.2,1.1,19.8,0.62,0.64,0.02,1.9519,1.9433,14.7319,16.8634,-,5,1
2,136668,0.532923,0.532929,310,918,1.1,2.3,1.6,17.0,0.82,0.78,0.04,1.9095,1.9396,30.2847,15.8391,-,5,0
3,141414,0.335690,0.335669,278,919,0.8,1.5,0.6,2.6,0.41,0.37,0.04,3.0467,2.9930,14.7504,71.8907,-,6,0
4,142794,0.470787,0.470802,270,63,1.0,2.3,1.8,11.9,0.72,0.72,0.00,2.1848,2.1851,16.4880,16.3626,-,5,0


In [10]:
blazhko_can.to_csv('../outputs/lc_MASTER_bcan.csv', index=False)